In [1]:
import pandas as pd
import folium
import geopandas as gpd
import folium.plugins as plugins

In [2]:
df = pd.read_csv('msft_clean_df.csv')

In [3]:
df.head()

,date,ip,loc,lat,long,Date,Day
0,2021-06-23 16:50:03 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 16:50:03+00:00,2021-06-23
1,2021-06-23 19:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 19:19:12+00:00,2021-06-23
2,2021-06-23 21:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 21:19:12+00:00,2021-06-23
3,2021-06-23 23:41:19 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 23:41:19+00:00,2021-06-23
4,2021-06-24 02:10:18 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-24 02:10:18+00:00,2021-06-24


In [4]:
df.columns

Index(['date', 'ip', 'loc', 'lat', 'long', 'Date', 'Day'], dtype='object')

In [5]:
df.dtypes

date     object
ip       object
loc      object
lat     float64
long    float64
Date     object
Day      object
dtype: object

In [6]:
# df["lat_long"] = list(zip(df["lat"], df["long"]))

In [7]:
df.head()

,date,ip,loc,lat,long,Date,Day
0,2021-06-23 16:50:03 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 16:50:03+00:00,2021-06-23
1,2021-06-23 19:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 19:19:12+00:00,2021-06-23
2,2021-06-23 21:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 21:19:12+00:00,2021-06-23
3,2021-06-23 23:41:19 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 23:41:19+00:00,2021-06-23
4,2021-06-24 02:10:18 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-24 02:10:18+00:00,2021-06-24


In [8]:
# lat_long = df['lat_long']

In [9]:
# dates = pd.to_datetime(df['Date'])

# function to generate base map, has default values for zoom and tiles
def generateBaseMap(loc, zoom=12, min_zoom=0, max_zoom=18, zoom_start=10, tiles='Stamen Toner', crs='ESPG2263'):
    '''
    Function that generates a Folium base map
    Input location lat/long
    Zoom level default 12
    Tiles default to Stamen Toner
    CRS default 2263 for NYC
    '''
    return folium.Map(location=loc, 
                      control_scale=True, 
                      zoom_start=zoom,
                      tiles=tiles)
  
mi = [42.279594, -83.732124] # generic nyc lat/lon in list format
base_map = generateBaseMap(mi) # pass lat/lon to function
base_map

from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime


#create base map
# mi_base_map = folium.Map(location = [42.279594, -83.732124], control_scale = True, zoom_start = 10, tiles = 'Stamen Toner')

#create the heatmap from list
HeatMap(data = df['lat_long'], radius=12).add_to(base_map)

base_map.save('bike_station_HeatMap.html')

# call map 
base_map

In [10]:
lat_long_list = []
for i in df['Date'].unique():
    temp=[]
    for index, instance in df[df['Date'] == i].iterrows():
        temp.append([instance['lat'],instance['long']])
    lat_long_list.append(temp)

In [11]:
lat_long_list[0][0]

[42.2709, -83.7282]

In [12]:
type(lat_long_list[0][0])

list

In [13]:
df['Date']= pd.to_datetime(df['Date'])
#creating a time index
time_index = []
for i in df['Date'].unique():
    time_index.append(i)
#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [14]:
# time_index

In [15]:
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

#Choosing the map type 
m = folium.Map(location=[42.2808, -83.7430],zoom_start = 11,control_scale=True, tiles='openstreetmap',attr="Stadia.AlidadeSmoothDark")

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=10,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)
# Display the map
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m

In [16]:
m_n = folium.Map(df[['lat', 'long']].mean().values.tolist())

for lat, lon in zip(df['lat'], df['long']):
    folium.Marker([lat, lon]).add_to(m_n)

In [17]:
sw = df[['lat', 'long']].min().values.tolist()
ne = df[['lat', 'long']].max().values.tolist()

m_n.fit_bounds([sw, ne]) 
m_n

In [18]:
from folium.plugins import MarkerCluster
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, HeatMapWithTime
import math

In [19]:
m_2 = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
for idx, row in df.iterrows():
    Marker([row['lat'], row['long']]).add_to(m_2)

In [20]:
m_3 = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
mc = MarkerCluster()
for idx, row in df.iterrows():
    if not math.isnan(row['long']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['long']]))
m_3.add_child(mc)

# Adds tool to the top right
from folium.plugins import MeasureControl
m_3.add_child(MeasureControl())

# Display the map
m_3

## Foursquare API work

In [21]:
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import requests

In [22]:
CLIENT_ID = 'JKHM1V3IJMGJTDH13VB2UKKYNLHN0N3XG4UENX0JEKBUZSMP' # your Foursquare ID
CLIENT_SECRET = 'CTHTUE1JSINGUNVXSDL0JGPO1CRKOE312RYYA21LEBKE5ZSA' # your Foursquare Secret
VERSION = '20220208'
LIMIT = 100

In [23]:
df.head()

,date,ip,loc,lat,long,Date,Day
0,2021-06-23 16:50:03 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 16:50:03+00:00,2021-06-23
1,2021-06-23 19:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 19:19:12+00:00,2021-06-23
2,2021-06-23 21:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 21:19:12+00:00,2021-06-23
3,2021-06-23 23:41:19 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 23:41:19+00:00,2021-06-23
4,2021-06-24 02:10:18 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-24 02:10:18+00:00,2021-06-24


In [24]:
ndf = df[['loc','lat','long']]

In [25]:
nndf = ndf.drop_duplicates()

In [26]:
# pd.options.display.float_format = "{:,.2f}".format
nndf

,loc,lat,long
0,"Ann Arbor, Michigan",42.2709,-83.7282
44,"Detroit, Michigan",37.7510,-83.1087
73,"Ann Arbor, Michigan",42.2709,-83.7068
80,"Ann Arbor, Michigan",42.7069,-83.7068
81,"Ann Arbor, Michigan",42.2804,-83.7068
109,"Hamtramck, Michigan",42.4122,-83.0583
110,"Detroit, Michigan",42.3914,-83.1500
154,"Astoria, New York",40.7706,-73.9265
168,"None, New Jersey",40.7064,-74.2111
233,"None, Tennessee",36.1671,-86.7844


In [27]:
nndf["long_lat"] = list(zip(nndf["lat"], nndf["long"]))


<ipython-input-27-023427462055>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nndf["long_lat"] = list(zip(nndf["lat"], nndf["long"]))


In [28]:
nndf

,loc,lat,long,long_lat
0,"Ann Arbor, Michigan",42.2709,-83.7282,"(42.2709, -83.7282)"
44,"Detroit, Michigan",37.7510,-83.1087,"(37.751, -83.1087)"
73,"Ann Arbor, Michigan",42.2709,-83.7068,"(42.2709, -83.7068)"
80,"Ann Arbor, Michigan",42.7069,-83.7068,"(42.7069, -83.7068)"
81,"Ann Arbor, Michigan",42.2804,-83.7068,"(42.2804, -83.7068)"
109,"Hamtramck, Michigan",42.4122,-83.0583,"(42.4122, -83.0583)"
110,"Detroit, Michigan",42.3914,-83.1500,"(42.3914, -83.15)"
154,"Astoria, New York",40.7706,-73.9265,"(40.7706, -73.9265)"
168,"None, New Jersey",40.7064,-74.2111,"(40.7064, -74.2111)"
233,"None, Tennessee",36.1671,-86.7844,"(36.1671, -86.7844)"


In [29]:
concat_df = pd.DataFrame()

In [137]:
concat_df2 = pd.DataFrame()

In [30]:
for index, row in nndf.iterrows():
    url = "https://api.foursquare.com/v3/places/nearby?ll={},{}&limit=20".format(round(row['lat'], 2),round(row['long'], 2))
#     print(url)
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3chVTJib0Z11IA8qFisvs8p7dkOJ6ky0WEbGTZ9FQPqc="
    }
    result = requests.request("GET", url, headers=headers).json()
    result_df = json_normalize(result['results'])
    result_df = result_df.rename(columns={"geocodes.main.latitude": "lat", "geocodes.main.longitude": "long", "location.formatted_address": "address"})
#     print(result_df)
    concat_df = concat_df.append(result_df)

<ipython-input-30-93558f7678ae>:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  result_df = json_normalize(result['results'])


In [157]:
user_input = input()

chinese


In [177]:
for index, row in nndf.iterrows():
    url = "https://api.foursquare.com/v3/places/search?query={}&ll={},{}&radius=50000&limit=5".format(user_input, row['lat'], row['long'])
#     print(url)
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3chVTJib0Z11IA8qFisvs8p7dkOJ6ky0WEbGTZ9FQPqc="
    }
    result = requests.request("GET", url, headers=headers).json()
    result_df = json_normalize(result['results'])
    result_df = result_df.rename(columns={"geocodes.main.latitude": "lat", "geocodes.main.longitude": "long", "location.formatted_address": "address"})
#     print(result_df)
    concat_df2 = concat_df.append(result_df)

<ipython-input-177-b83463b07424>:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  result_df = json_normalize(result['results'])


In [161]:
concat_df

,fsq_id,categories,chains,distance,name,timezone,lat,long,location.address,location.country,...,location.region,related_places.parent.fsq_id,related_places.parent.name,geocodes.front_door.latitude,geocodes.front_door.longitude,location.neighborhood,location.po_box,location.address_extended,related_places.children,category
0,4ba43fa9f964a520ec8f38e3,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...",[],156,The Metal Frat,America/Detroit,42.271297,-83.729266,907 Lincoln Ave,US,...,MI,4c0eff8c15c2ef3bc3581b08,University of Michigan,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 10039, 'name': 'Music Venue', 'icon': ..."
1,4e51d96418a8af30fda6ba1f,"[{'id': 12043, 'name': 'Fraternity House', 'ic...",[],158,Delta Kappa Epsilon,America/Detroit,42.269721,-83.731891,1004 Olivia Ave,US,...,MI,4c0eff8c15c2ef3bc3581b08,University of Michigan,42.269699,-83.731770,[Minerva Road Homeowners Association],NaN,NaN,NaN,"[{'id': 12043, 'name': 'Fraternity House', 'ic..."
2,595eca523731ee6054027f77,"[{'id': 11079, 'name': 'Carpet and Flooring Co...",[],1902,Floor Trader,America/Detroit,42.253068,-83.733442,2301 S Industrial Hwy,US,...,MI,NaN,NaN,42.253019,-83.733578,[Briarwood],NaN,NaN,NaN,"[{'id': 11079, 'name': 'Carpet and Flooring Co..."
3,4c8d250cd5049c74ff844d6a,"[{'id': 16047, 'name': 'Sculpture Garden', 'ic...",[],317,The Rock,America/Detroit,42.272453,-83.728026,1584 Hill St,US,...,MI,NaN,NaN,NaN,NaN,[Oxbridge],NaN,NaN,NaN,"[{'id': 16047, 'name': 'Sculpture Garden', 'ic..."
4,4bc536daccbcef3b9f54e6d2,"[{'id': 12043, 'name': 'Fraternity House', 'ic...",[],179,Chem Frat,America/Detroit,42.270091,-83.732183,1319 Cambridge Rd,US,...,MI,4c0eff8c15c2ef3bc3581b08,University of Michigan,42.270101,-83.732170,[Minerva Road Homeowners Association],NaN,NaN,NaN,"[{'id': 12043, 'name': 'Fraternity House', 'ic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,4b9c3834f964a5202b5636e3,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",[{'id': 'ab4ae740-d68a-012e-5619-003048cad9da'...,3553,Wendy's,America/Detroit,41.958248,-83.668454,549 Tecumseh St,US,...,MI,NaN,NaN,41.958374,-83.668467,NaN,NaN,NaN,NaN,"[{'id': 13145, 'name': 'Fast Food Restaurant',..."
15,4d1cb53d4ff98cfa3a05be3e,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",[{'id': 'ab4b8010-d68a-012e-5619-003048cad9da'...,3610,Taco Bell,America/Detroit,41.958295,-83.667761,537 Tecumseh St,US,...,MI,NaN,NaN,41.958356,-83.667798,NaN,NaN,NaN,NaN,"[{'id': 13145, 'name': 'Fast Food Restaurant',..."
16,4c364d022c8020a1ab168700,"[{'id': 13099, 'name': 'Chinese Restaurant', '...",[],3105,Chinese Sizzling Restaurant,America/Detroit,41.957268,-83.673716,119 Waterstradt Commerce Dr,US,...,MI,NaN,NaN,41.956949,-83.674091,NaN,NaN,Ste 4,NaN,"[{'id': 13099, 'name': 'Chinese Restaurant', '..."
17,4bc352f8dce4eee14a8c719d,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[],3169,Holiday Inn Dundee,America/Detroit,41.961586,-83.674954,100 Whitetail Dr,US,...,MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref..."


In [178]:
concat_df2

,fsq_id,categories,chains,distance,name,timezone,lat,long,location.address,location.country,...,location.region,related_places.parent.fsq_id,related_places.parent.name,geocodes.front_door.latitude,geocodes.front_door.longitude,location.neighborhood,location.po_box,location.address_extended,related_places.children,category
0,4ba43fa9f964a520ec8f38e3,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...",[],156,The Metal Frat,America/Detroit,42.271297,-83.729266,907 Lincoln Ave,US,...,MI,4c0eff8c15c2ef3bc3581b08,University of Michigan,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 10039, 'name': 'Music Venue', 'icon': ..."
1,4e51d96418a8af30fda6ba1f,"[{'id': 12043, 'name': 'Fraternity House', 'ic...",[],158,Delta Kappa Epsilon,America/Detroit,42.269721,-83.731891,1004 Olivia Ave,US,...,MI,4c0eff8c15c2ef3bc3581b08,University of Michigan,42.269699,-83.731770,[Minerva Road Homeowners Association],NaN,NaN,NaN,"[{'id': 12043, 'name': 'Fraternity House', 'ic..."
2,595eca523731ee6054027f77,"[{'id': 11079, 'name': 'Carpet and Flooring Co...",[],1902,Floor Trader,America/Detroit,42.253068,-83.733442,2301 S Industrial Hwy,US,...,MI,NaN,NaN,42.253019,-83.733578,[Briarwood],NaN,NaN,NaN,"[{'id': 11079, 'name': 'Carpet and Flooring Co..."
3,4c8d250cd5049c74ff844d6a,"[{'id': 16047, 'name': 'Sculpture Garden', 'ic...",[],317,The Rock,America/Detroit,42.272453,-83.728026,1584 Hill St,US,...,MI,NaN,NaN,NaN,NaN,[Oxbridge],NaN,NaN,NaN,"[{'id': 16047, 'name': 'Sculpture Garden', 'ic..."
4,4bc536daccbcef3b9f54e6d2,"[{'id': 12043, 'name': 'Fraternity House', 'ic...",[],179,Chem Frat,America/Detroit,42.270091,-83.732183,1319 Cambridge Rd,US,...,MI,4c0eff8c15c2ef3bc3581b08,University of Michigan,42.270101,-83.732170,[Minerva Road Homeowners Association],NaN,NaN,NaN,"[{'id': 12043, 'name': 'Fraternity House', 'ic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4c364d022c8020a1ab168700,"[{'id': 13099, 'name': 'Chinese Restaurant', '...",[],2767,Chinese Sizzling Restaurant,America/Detroit,41.957268,-83.673716,119 Waterstradt Commerce Dr,US,...,MI,NaN,NaN,41.956949,-83.674091,NaN,NaN,Ste 4,NaN,NaN
1,4bc1145b4cdfc9b6d81e9421,"[{'id': 13099, 'name': 'Chinese Restaurant', '...",[],27094,Golden Dragon American Chinese,America/New_York,41.717562,-83.623733,5624 Secor Rd,US,...,OH,NaN,NaN,41.717681,-83.623858,NaN,NaN,NaN,NaN,NaN
2,4b96e869f964a52003ed34e3,"[{'id': 13099, 'name': 'Chinese Restaurant', '...",[],33133,Lucky 7 Chinese Food,America/Detroit,42.247557,-83.641132,1777 Washtenaw Rd,US,...,MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4bef35bfd1b7d13a337fcdec,"[{'id': 13099, 'name': 'Chinese Restaurant', '...",[],39587,Empire Chinese Restaurant,America/New_York,41.636375,-83.488853,2502 Navarre Ave,US,...,OH,NaN,NaN,41.636472,-83.488833,[Ryan],NaN,NaN,NaN,NaN


In [176]:
concat_df2.name

0                   The Metal Frat
1              Delta Kappa Epsilon
2                     Floor Trader
3                         The Rock
4                        Chem Frat
                 ...              
5    Great Wall Chinese Restaurant
6             Lucky Garden Chinese
7                       Jing Chuan
8     Chinatown Chinese Restaurant
9                Oriental Sizzling
Name: name, Length: 252, dtype: object

In [47]:
concat_df.columns

Index(['fsq_id', 'categories', 'chains', 'distance', 'name', 'timezone', 'lat',
       'long', 'location.address', 'location.country', 'location.cross_street',
       'location.dma', 'address', 'location.locality', 'location.postcode',
       'location.region', 'related_places.parent.fsq_id',
       'related_places.parent.name', 'geocodes.front_door.latitude',
       'geocodes.front_door.longitude', 'location.neighborhood',
       'location.po_box', 'location.address_extended',
       'related_places.children'],
      dtype='object')

In [76]:
new_map = folium.Map(location=[42.2808, -83.7430], zoom_start=15)
# add a red circle marker to represent each visited locations
for lat, long in zip(nndf.lat, nndf.long):
    folium.features.CircleMarker(
        [lat, long],
        radius=10,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(new_map)
# # add all venues as blue circle markers
for lat, long, label in zip(concat_df.lat, concat_df.long, concat_df.name):
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(new_map)

In [148]:
new_map = folium.Map(location=[42.2808, -83.7430], zoom_start=15)
# add a red circle marker to represent each visited locations
for lat, long in zip(nndf.lat, nndf.long):
    folium.features.CircleMarker(
        [lat, long],
        radius=10,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(new_map)
# # add all venues as blue circle markers
for lat, long, label in zip(concat_df.lat, concat_df.long, concat_df.categories):
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(new_map)

In [149]:
new_map

In [172]:
new_map2 = folium.Map(location=[42.2808, -83.7430], zoom_start=15)
# add a red circle marker to represent each visited locations
for lat, long in zip(nndf.lat, nndf.long):
    folium.features.CircleMarker(
        [lat, long],
        radius=10,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(new_map2)
# # add all venues as blue circle markers
for lat, long, label in zip(concat_df2.lat, concat_df2.long, concat_df2.name):
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(new_map2)

In [173]:
new_map2